In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.ensemble import GradientBoostingRegressor
from joblib import dump,load

In [2]:
#读数据标准化划分数据集
df = pd.read_csv('../my_data.csv')
X = df.iloc[:, 1:58]
Y = df.iloc[:, 58:]
X_to_select = ['A+B_Ionic_radius(Å)', 'A+X_Ionic_radius(Å)', 'B+X_Ionic_radius(Å)', 'A/B_Ionic_radius(Å)', 
               'A/X_Ionic_radius(Å)', 'B/X_Ionic_radius(Å)', 'Tolerance factor', 'Octahedral factor', 
              'B_Ionization energy(kJ/mol)', 'X_Ionization energy(kJ/mol)', 'A_Electron_affinity(kJ/mol)', 
              'B_Electron_affinity(kJ/mol)', 'B_Valence_electrons', 'A_Boiling_point(K)', 'B_Boiling_point(K)', 
              'B_Melting_point(K)', 'A_Density(g/cc)', 'B_Density(g/cc)', 'B_Atomic_weight(u)', 'X_Atomic_weight(u)', 
              'B_Heat_of_fusion(kJ/mol)', 'X_Heat_of_fusion(kJ/mol)', 'B_Heat_of_vap(kJ/mol)', 'B_Electronegativity', 
              'X_Electronegativity']
X=X[X_to_select]
Y=Y['SLME @ 5um']
names=df.columns
names=['SHAP value of '+x for x in names]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=10)
print("X_train's shape is", X_train.shape,"; y_train's shape is", y_train.shape)
print("X_test's shape is", X_test.shape,"; y_test's shape is",y_test.shape)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_stand = scaler.transform(X_train)
X_test_stand = scaler.transform(X_test)

X_train's shape is (390, 25) ; y_train's shape is (390,)
X_test's shape is (98, 25) ; y_test's shape is (98,)


In [3]:
##gridsearch and cv select model##
param_grid = {
    'n_estimators': [20, 40, 60, 80, 100, 300, 500, 1000],
    'max_depth': range(1, 10),
    'min_samples_split': [1, 2, 3, 5],
    'learning_rate': [0.001, 0.01, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'min_samples_leaf': [1, 2, 3, 4],
    'max_features': ['auto', 'sqrt'],
    'alpha': [0, 0.01, 0.1, 1.0],
}


In [4]:
gb_regressor = GradientBoostingRegressor()
model = GridSearchCV(estimator =gb_regressor,scoring='neg_root_mean_squared_error', param_grid = param_grid,cv=5, verbose=3,n_jobs =-1,return_train_score=True)
model.fit(X_train_stand, y_train)
print(model.cv_results_)
dfresult=pd.DataFrame(model.cv_results_)
dfresult.to_csv('SLME_gbdtVbm2.csv')
print(dfresult)
print(model.best_params_)
print(model.scorer_)
print(model.best_score_)
bestmodel=model.best_estimator_

Fitting 5 folds for each of 110592 candidates, totalling 552960 fits


D:\App\Anaconda3\envs\test\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
345600 fits failed out of a total of 552960.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
138240 fits failed with the following error:
Traceback (most recent call last):
  File "D:\App\Anaconda3\envs\test\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\App\Anaconda3\envs\test\lib\site-packages\sklearn\ensemble\_gb.py", line 525, in fit
    self._check_params()
  File "D:\App\Anaconda3\envs\test\lib\site-packages\sklearn\ensemble\_gb.py", line 330, in _check_params
    raise ValueError("alpha must be in (0.0, 1.0) but was %r" % sel

        mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0            0.000800      0.000400              0.0             0.0   
1            0.000400      0.000490              0.0             0.0   
2            0.000202      0.000404              0.0             0.0   
3            0.000400      0.000490              0.0             0.0   
4            0.000402      0.000492              0.0             0.0   
...               ...           ...              ...             ...   
110587       0.000200      0.000400              0.0             0.0   
110588       0.001001      0.000002              0.0             0.0   
110589       0.000501      0.000447              0.0             0.0   
110590       0.000000      0.000000              0.0             0.0   
110591       0.000301      0.000602              0.0             0.0   

       param_alpha param_learning_rate param_max_depth param_max_features  \
0                0               0.001               1    